## Testing vector_store.py module

for handling local sqlite db and weaviate collections

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import my_mirror_on_cloud.vector_store as vs
import my_mirror_on_cloud.text_generation as tg

In [ ]:
# Initialize the local catalog store
store = vs.LocalCatalogStore(db_path="../data/catalogue_v1.db")
store

In [ ]:
image1 =  "../data/h-and-m-personalized-fashion-recommendations/images/093/0930409001.jpg"

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig, axes = plt.subplots(1, 1, figsize=(3, 3))

img = mpimg.imread(image1)
axes.imshow(img)
axes.axis('off')  

plt.tight_layout()
plt.show()

In [ ]:
from pprint import pprint

description = tg.analyze_clothing_image(
        image1,
        model_name="qwen2.5vl:7b",
        prompt_type="description_only",
        max_width=256,
    )
pprint(description)

In [ ]:
tags=tg.get_tags_from_analysis(analysis=description)

processing_status = tg.get_processing_status(analysis=description)

processing_status, tags

In [ ]:
image_id = store.insert_image(image1, tags, processing_status, embedding=None)

In [ ]:
store.close()

## Analyzing the full catalogue!



In [ ]:
# Get images
from pathlib import Path
image_paths = Path('../data/h-and-m-personalized-fashion-recommendations/images').rglob('*.jpg')
image_list = list(image_paths)
print(f"Found {len(image_list)} images, {image_list[0]}")

In [ ]:
store = vs.LocalCatalogStore(db_path="../data/catalogue_v1.db")

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
model_name = "qwen2.5vl:7b"
prompt_type = "description_only"

for img_path in tqdm(image_list):
    img_path_str = str(img_path)
    model_name = "qwen2.5vl:7b"
    
    if store.is_model_processed(img_path_str, model_name):
        continue

    description = tg.analyze_clothing_image(
        img_path_str,
        model_name=model_name,
        max_width=256,
        prompt_type=prompt_type,
    )
    tags = tg.get_tags_from_analysis(analysis=description)
    processing_status = tg.get_processing_status(analysis=description)
    store.insert_image(img_path_str, tags, processing_status, embedding=None)
    
    if len(store.get_all_images()) % 100 == 0:
        print(f"Processed {len(store.get_all_images())} images so far.")

